In [12]:
import csv
import plotly.graph_objects as go


def read_calibration_data(data_path):
    with open(data_path, 'r') as file:
        csv_reader = csv.DictReader(file)
        data = [row for row in csv_reader]
    return data

def apply_offset(data, x, y):
    for row in data:
        row['Prediction'] = str(float(row['Prediction']) + x)
        row['Measurement'] = str(float(row['Measurement']) + y)
    return data

def plot_calibration_data(data):
    predictions = [float(row['Prediction']) for row in data]
    mesurements = [float(row['Measurement']) for row in data]
    fig = go.Figure()
    fig.add_trace(go.Scatter( x=predictions, y=mesurements, name="yaxis1 data", yaxis="y1" ))
    # square aspect ratio
    fig.update_layout(
        autosize=False,
        width=500,
        height=500,
        yaxis=dict(
            scaleanchor="x",
            scaleratio=1,
        ),
    )
    # add ticks every 0.1
    fig.update_xaxes(dtick=0.1, title_text="Prediction")
    fig.update_yaxes(dtick=0.1, title_text="Measurement")
    fig.show()

def correct_prediction(prediction, data):
    """
    Corrects the prediction interpolating the data
    """
    # prediction is out of range
    last_prediction = float(data[-1]['Prediction'])
    if prediction > last_prediction:
        return float(data[-1]['Measurement']) + (prediction - last_prediction)

    # Find the two closest points
    for i in range(len(data)):
        if prediction < float(data[i]['Prediction']):
            break
    # Interpolate
    x1 = float(data[i-1]['Prediction'])
    x2 = float(data[i]['Prediction'])
    y1 = float(data[i-1]['Measurement'])
    y2 = float(data[i]['Measurement'])
    return y1 + (y2-y1)/(x2-x1) * (prediction-x1)

def correct_gcode(gcode_path, data=data):
    with open(gcode_path, 'r') as file:
        gcode = file.readlines()
    for i in range(len(gcode)):
        if gcode[i].startswith('G1 Z'):
            prediction_str = gcode[i].split(' ')[1][1:]
            prediction = float(prediction_str)
            gcode[i] = gcode[i].replace(prediction_str, str(correct_prediction(prediction, data)))
    # create new file
    with open(gcode_path[:-6] + '_corrected.gcode', 'w') as file:
        file.writelines(gcode)


In [24]:
data_path = "../data/test1.csv"
gcode_path = "../data/mount2.gcode"
data = read_calibration_data(data_path)
data = apply_offset(data, x=0.1, y=0)
plot_calibration_data(data)
correct_gcode(gcode_path, data)
